In [ ]:
# import os

# This is needed due to zipkin issues related to protobuf.
# os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

from pathlib import Path
import sys

import dotenv

# Add base dir to path to be able to access test folder.
base_dir = Path().cwd().parent.parent.resolve()
if str(base_dir) not in sys.path:
    print(f"Adding {base_dir} to sys.path")
    sys.path.append(str(base_dir))


dotenv.load_dotenv()

# Deps
# ! pip install opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp

# Install zipkin python package:
# ! pip install opentelemetry-exporter-zipkin-proto-http
# Or other exporters
! pip install opentelemetry-exporter-prometheus

# Start the zipkin docker container:
# ! docker run --rm -d -p 9411:9411 --name zipkin openzipkin/zipkin

# Stop the zipkin docker container:
# ! docker stop $(docker ps -a -q --filter ancestor=openzipkin/zipkin)

In [ ]:
from opentelemetry.exporter.zipkin.json import ZipkinExporter
from trulens.apps.custom import TruCustomApp
from trulens.core.session import TruSession
from trulens.feedback.dummy.endpoint import DummyEndpoint

from examples.dev.dummy_app.app import DummyApp

tru = TruSession(
    _experimental_otel_exporter=ZipkinExporter(
        endpoint="http://localhost:9411/api/v2/spans"
    )
)


TruSession().experimental_enable_feature("otel_tracing")

# Need to create dummy endpont for the instrumentation of the DummyAPI to
# happen.

DummyEndpoint()


# tru.reset_database()
# run_dashboard(force=True, port=8080)

In [ ]:
# Create custom app:
ca = DummyApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
)

In [ ]:
# Nested context managers.
with ta as recorder1:
    ca.llm.generate("this is the outer memory remember")
    with ta as recorder2:
        ca.memory.remember("this is the inner memory remember")
        ca.respond_to_query("hello")

In [ ]:
from trulens.experimental.otel_tracing.core.trace import get_tracer

In [ ]:
get_tracer().spans

In [ ]:
print(recorder1.records[2])